# Chem

> Chemistry related functions

In [ ]:
#| default_exp chem

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from chem_templates.imports import *
from chem_templates.utils import *

import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem, rdMolDescriptors, Descriptors
from rdkit.Chem.FilterCatalog import FilterCatalog, ExclusionList, FilterCatalogEntry, \
SmartsMatcher, FilterCatalogParams
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

## RDKit i/o

Functions for converting between SMILES strings and RDKit mol objects

In [ ]:
#| export
def to_mol(smile: str) -> Union[Chem.Mol, None]:
    try:
        mol = Chem.MolFromSmiles(smile)
        Chem.SanitizeMol(mol)
    except:
        mol = None
        
    return mol

def to_smile(mol: Chem.Mol) -> str:
    smile = Chem.MolToSmiles(mol)
    return smile

def to_kekule(smile: str) -> str:
    return Chem.MolToSmiles(to_mol(smile), kekuleSmiles=True)

def canon_smile(smile: str) -> str:
    try:
        return Chem.CanonSmiles(smile)
    except:
        return ''
    
def remove_stereo(mol: Chem.Mol) -> Chem.Mol:
    Chem.rdmolops.RemoveStereochemistry(mol)
    return mol

def remove_stereo_smile(smile: str) -> str:
    if '@' in smile:
        mol = to_mol(smile)
        mol = remove_stereo(mol)
        smile = to_smile(mol)
    return smile

In [ ]:
assert type(to_mol('CCC')) == Chem.Mol
assert type(to_smile(Chem.MolFromSmiles('CCC'))) == str

In [ ]:
#| export
class Molecule():
    def __init__(self, 
                 smile: str, 
                 data:  Optional[dict]=None):
        self.smile = canon_smile(smile)
        self.mol = to_mol(self.smile)
        self.valid = (self.mol is not None) and (self.smile != '')
        
        self.data = {}
        self.add_data(data)
            
    def add_data(self, data: Optional[dict]=None):
        if data is not None:
            self.data.update(data)

In [ ]:
#| export

def mol_func_wrapper(func: Callable[[Chem.Mol], Any]):
    return lambda x: func(x.mol)

In [ ]:
rdkit_function = Descriptors.MolLogP

smile = 'CCCCCC'
mol = to_mol(smile)
molecule = Molecule(smile)

wrapped_function = mol_func_wrapper(rdkit_function)

assert rdkit_function(mol) == wrapped_function(molecule)

In [ ]:
#| export

class Catalog():
    def __init__(self, catalog: FilterCatalog):
        self.catalog = catalog
        self.filter_names = [self.catalog.GetEntryWithIdx(i).GetDescription() 
                      for i in range(self.catalog.GetNumEntries())]
        
    def has_match(self, molecule: Molecule) -> bool:
        return self.catalog.HasMatch(molecule.mol)
    
    def get_matches(self, molecule: Molecule) -> list[str]:
        matches = [i.GetDescription() for i in self.catalog.GetMatches(molecule.mol)]
        return matches
    
    @classmethod
    def from_smarts(cls, smarts: list[str]):
        catalog = FilterCatalog()
        for s in smarts:
            catalog.AddEntry(FilterCatalogEntry(s, SmartsMatcher(s,s)))
            
        return cls(catalog)
    
    @classmethod
    def from_params(cls, params: FilterCatalogParams.FilterCatalogs):
        catalog = FilterCatalog(params)
        return cls(catalog)

In [ ]:
smarts = [
    '[*]-[#6]1:[#6]:[#6](-[#0]):[#6]:[#6](-[*]):[#6]:1',
    '[*]-[#6]1:[#6]:[#6](-[*]):[#6]:[#6]:[#6]:1',
    '[*]-[#6]1:[#6]:[#6]:[#6]:[#6]:[#6]:1',
    '[*]-[#6]1:[#6]:[#6](-[#7]-[*]):[#6]:[#6]:[#6]:1',
    '[#6]1:[#6]:[#7]:[#6]:[#6]:[#6]:1'
]

catalog = Catalog.from_smarts(smarts)

smiles = [
    'c1ccccc1',
    'Cc1cccc(NCc2ccccc2)c1'
]

molecules = [Molecule(i) for i in smiles]

assert not catalog.has_match(molecules[0])
assert catalog.has_match(molecules[1])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()